In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
import matplotlib.pyplot as plt
from lib import *
import numpy as np
import pandas as pd

In [ ]:
for name in ["Ug","Eu","Cs","Co"]:
    x,y=read(f"mca/Ge_{name}.txt")
    config=readConfig(f"mca/Ge_{name}.mcd")
    y*=600/config["REALTIME"]
    plt.plot(x,y,label=name)
    if name=="Ug":
        name="Untergrund"
    plt.title(f"{name} Spektrum des Ge Detektor")
    plt.ylabel("Ereignisse in 10 Minuten")
    plt.xlabel("Bin #")
    plt.show()

In [ ]:
for name in ["Ug","Eu","Cs","Co"]:
    x,y=read(f"mca/Sz_{name}.txt")
    config=readConfig(f"mca/Ge_{name}.mcd")
    y*=600/config["REALTIME"]
    plt.plot(x,y,label=name)
    if name=="Ug":
        name="Untergrund"
    plt.title(f"{name} Spektrum des Szintillationsdetektor")
    plt.ylabel("Ereignisse in 10 Minuten")
    plt.xlabel("Bin #")
    plt.show()

In [ ]:
def gauss_linear(x,p,sd,u,a,b):
    return p*np.exp(-((x-u)/(np.sqrt(2)*sd))**2)+a*x+b
def fitpeak(x,y,a,b,label="Anpassung",gauss=False):
    x=x[a:b]
    y=y[a:b]
    guess=[
        np.max(y)-np.min(y),
        (b-a)/10,
        x[y.tolist().index(np.max(y))],
        0,
        np.min(y)]
    param,err,chi= curve_fit(gauss_linear, x, y, np.maximum(1,np.sqrt(y)), guess) if gauss else poisson_fit(gauss_linear, x, y,guess)
    plt.plot(x,gauss_linear(x,*param),label=label)
    return [param,err,[chi]]

In [ ]:
cols=["energie","h","sd","u","a","b","d_h","d_sd","d_u","d_a","d_b","chi2"]
EnergieCal=pd.DataFrame(columns=cols)

ux,uy=read("mca/Sz_Ug.txt")
x,y=read("mca/Sz_Cs.txt")
y=np.maximum(0,y-uy)
plt.plot(x,y,".",color=(0,0,0))
res=[]
res.append([[661.660]]+fitpeak(x,y,3200,3880,"661.660 keV"))
plt.yscale("log")
plt.title("Cs Spektrum minus Untergrund")
plt.ylabel("Ereignisse in 10 Minuten")
plt.xlabel("Bin #")
plt.legend()
plt.show()
EnergieCal=pd.concat([EnergieCal]+[
    pd.DataFrame([np.concatenate(res[i])],columns=cols)
    for i in range(len(res))],ignore_index=True)

In [ ]:
ux,uy=read("mca/Sz_Ug.txt")
x,y=read("mca/Sz_Co.txt")
y=np.maximum(0,y-uy)
plt.plot(x,y,".",color=(0,0,0))
res=[]
res.append([[1173.237]]+fitpeak(x,y,5900,6600,"1173.237 keV"))
res.append([[1332.516]]+fitpeak(x,y,6700,7500,"1332.516 keV"))
plt.yscale("log")
plt.title("Co Spektrum minus Untergrund")
plt.ylabel("Ereignisse in 10 Minuten")
plt.xlabel("Bin #")
plt.legend()
plt.show()
EnergieCal=pd.concat([EnergieCal]+[
    pd.DataFrame([np.concatenate(res[i])],columns=cols)
    for i in range(len(res))],ignore_index=True)

In [ ]:
ux,uy=read("mca/Sz_Ug.txt")
x,y=read("mca/Sz_Eu.txt")
y=np.maximum(0,y-uy)
plt.plot(x,y,".",color=(0,0,0))
res=[]
res.append([[121.7825]]+fitpeak(x,y,600,750,"121.7825 keV"))
res.append([[244.6989]]+fitpeak(x,y,1200,1500, "244.6989 keV"))
res.append([[344.281]]+fitpeak(x,y,1700,2100, "344.281 keV"))
res.append([[778.903]]+fitpeak(x,y,4000,4400,"778.903 keV"))
res.append([[964.131]]+fitpeak(x,y,4900,5400, "964.131 keV"))
res.append([[1112.116]]+fitpeak(x,y,5500,6200, "1112.116 keV"))
res.append([[1408.011]]+fitpeak(x,y,7100,7800, "1408.011 keV"))
plt.yscale("log")
plt.title("Eu Spektrum minus Untergrund")
plt.ylabel("Ereignisse in 10 Minuten")
plt.xlabel("Bin #")
plt.legend()
plt.show()
EnergieCal=pd.concat([EnergieCal]+[
    pd.DataFrame([np.concatenate(res[i])],columns=cols)
    for i in range(len(res))],ignore_index=True)

In [ ]:
e, b, Db, sd, Dsd= np.array(EnergieCal[["energie","u","d_u","sd","d_sd"]]).T
def lin(x,a,b):
    return a*x+b
res=curve_fit(lin,e,b,np.sqrt((Db)**2+4))
plt.plot(e,b,".")
plt.plot(e,lin(e,*res[0]),"-")
plt.xlabel("Energie [keV]")
plt.ylabel("Bin #")
plt.title("Szintillator Energiekalibrierung")
plt.show()
print(res)

#compute FWHM
sd*=2*np.sqrt(2*np.log(2))
Dsd*=2*np.sqrt(2*np.log(2))
plt.errorbar(e,sd,Dsd,fmt=".")
plt.xlabel("Energie [keV]")
plt.ylabel("FWHM [bins]")
plt.title("Szintillator FWHM")
plt.show()
EnergieCal

In [ ]:
cols=["energie","h","sd","u","a","b","d_h","d_sd","d_u","d_a","d_b","chi2"]
EnergieCal2=pd.DataFrame(columns=cols)

ux,uy=read("mca/Ge_Ug.txt")
x,y=read("mca/Ge_Cs.txt")
y=np.maximum(0,y-uy)
plt.plot(x,y,".",color=(0,0,0))
res=[]
res.append([[661.660]]+fitpeak(x,y,3400,3460,"661.660 keV",gauss=True))
plt.yscale("log")
plt.title("Cs Spektrum minus Untergrund")
plt.ylabel("Ereignisse in 10 Minuten")
plt.xlabel("Bin #")
plt.legend()
plt.show()
EnergieCal2=pd.concat([EnergieCal2]+[
    pd.DataFrame([np.concatenate(res[i])],columns=cols)
    for i in range(len(res))],ignore_index=True)

In [ ]:
ux,uy=read("mca/Ge_Ug.txt")
x,y=read("mca/Ge_Co.txt")
y=np.maximum(0,y-uy)
plt.plot(x,y,".",color=(0,0,0))
res=[]
res.append([[1173.237]]+fitpeak(x,y,6000,6200,"1173.237 keV",gauss=True))
res.append([[1332.516]]+fitpeak(x,y,6700,7000,"1332.516 keV",gauss=True))
plt.yscale("log")
plt.title("Co Spektrum minus Untergrund")
plt.ylabel("Ereignisse in 10 Minuten")
plt.xlabel("Bin #")
plt.legend()
plt.show()
EnergieCal2=pd.concat([EnergieCal2]+[
    pd.DataFrame([np.concatenate(res[i])],columns=cols)
    for i in range(len(res))],ignore_index=True)

In [ ]:
ux,uy=read("mca/Ge_Ug.txt")
x,y=read("mca/Ge_Eu.txt")
y=np.maximum(0,y-uy)
plt.plot(x,y,".",color=(0,0,0))
res=[]
res.append([[121.7825]]+fitpeak(x,y,550,650,"121.7825 keV",gauss=True))
res.append([[244.6989]]+fitpeak(x,y,1200,1300,"244.6989 keV",gauss=True))
res.append([[344.281]]+fitpeak(x,y,1750,1800,"344.281 keV",gauss=True))
res.append([[411.115]]+fitpeak(x,y,2100,2150,"411.115 keV",gauss=True))
res.append([[443.976]]+fitpeak(x,y,2250,2350,"443.976 keV",gauss=True))
res.append([[778.903]]+fitpeak(x,y,4000,4100,"778.903 keV",gauss=True))
res.append([[867.388]]+fitpeak(x,y,4450,4600,"867.388 keV",gauss=True))
res.append([[964.131]]+fitpeak(x,y,4900,5100,"964.131 keV",gauss=True))
res.append([[1085.91]]+fitpeak(x,y,5600,5730,"1085.91 keV",gauss=True))
res.append([[1112.116]]+fitpeak(x,y,5730,5850,"1112.116 keV",gauss=True))
res.append([[1408.011]]+fitpeak(x,y,7200,7400,"1408.011 keV",gauss=True))
plt.yscale("log")
plt.title("Eu Spektrum minus Untergrund")
plt.ylabel("Ereignisse in 10 Minuten")
plt.xlabel("Bin #")
plt.legend()
plt.show()
EnergieCal2=pd.concat([EnergieCal2]+[
    pd.DataFrame([np.concatenate(res[i])],columns=cols)
    for i in range(len(res))],ignore_index=True)

In [ ]:
e, b, Db, sd, Dsd= np.array(EnergieCal2[["energie","u","d_u","sd","d_sd"]]).T
def lin(x,a,b):
    return a*x+b
res=curve_fit(lin,e,b,np.sqrt((Db)**2+1))
plt.plot(e,b,".")
plt.plot(e,lin(e,*res[0]),"-")
plt.xlabel("Energie [keV]")
plt.ylabel("Bin #")
plt.title("Ge Energiekalibrierung")
plt.show()
Ge_Energie_Cal=res
print(res)

#compute FWHM
sd=np.abs(sd)*2*np.sqrt(2*np.log(2))
Dsd*=2*np.sqrt(2*np.log(2))
res=curve_fit(lin,e,sd,np.sqrt((Dsd)**2+1))
plt.errorbar(e,sd,Dsd,fmt=".")
plt.plot(e,lin(e,*res[0]),"-")
plt.xlabel("Energie [keV]")
plt.ylabel("FWHM [bins]")
plt.title("Ge FWHM")
plt.show()
print(res)
EnergieCal2

In [ ]:
E,h,sd,dh,dsd=np.array(EnergieCal2[["energie","h","sd","d_h","d_sd"]])[3:].T
I=h*sd*np.sqrt(2*np.pi)
dI=np.sqrt(2*np.pi*((h*dsd)**2+(dh*sd)**2))/I[-1]*1000
I/=I[-1]/1000
Isoll=[1362,359,1275,107,148,621.6,199,693.4,475,649,1000]
nu=I/Isoll
dnu=dI/Isoll
plt.errorbar(E,nu,dnu,fmt=".")
plt.ylabel("Relative Effizienz")
plt.xlabel("Energie [keV]")
plt.title("Relative Effizienz vom Ge detektor")
plt.show()
print(pd.DataFrame(columns=["E","I","dI","Isoll","nu","dnu"],data=np.array([E,I,dI,Isoll,nu,dnu]).T).to_latex(index=False,float_format="%.2f"))

In [ ]:
x,y=read("mca/Rhein_lang.txt")
config=readConfig("mca/Rhein_lang.mcd")
x,ug_y=read("mca/Ug_lang.txt")
ug_config=readConfig("mca/Ug_lang.mcd")
ug_y*=config["REALTIME"]/ug_config["REALTIME"]

plt.plot(x,y,label=name)
plt.title("Spektrum der Bodenprobe")
plt.ylabel("Ereignisse")
plt.xlabel("Bin #")
plt.show()

plt.plot(x,ug_y)
plt.title("Untergrundspektrum")
plt.ylabel("Ereignisse")
plt.xlabel("Bin #")
plt.show()

y=np.maximum(0,y-ug_y)

cols=["h","sd","u","a","b","d_h","d_sd","d_u","d_a","d_b","chi2"]
Energie=pd.DataFrame(columns=cols)
plt.plot(x,y,".",color=(0,0,0))
res=[]
res.append(fitpeak(x,y,1100,1300,gauss=True))
res.append(fitpeak(x,y,1400,1600,gauss=True))
res.append(fitpeak(x,y,1700,1900,gauss=True))
res.append(fitpeak(x,y,2900,3100,gauss=True))
res.append(fitpeak(x,y,3100,3300,gauss=True))
res.append(fitpeak(x,y,7500,7800,gauss=True))
Energie=pd.concat([Energie]+[
    pd.DataFrame([np.concatenate(res[i])],columns=cols)
    for i in range(len(res))],ignore_index=True)
plt.title("Spektrum der Bodenprobe minus Untergrund")
plt.ylabel("Ereignisse")
plt.xlabel("Bin #")
plt.show()

In [ ]:
Energie

In [ ]:
for i in range(4):
    x,y=read(f"oszi/T000{i}.CSV",skip=17,cols=[0,1])
    x=x*1000000
    plt.plot(x,y)
    plt.title(["Ge Signal","Ge Signal verstärkt","Szintillator Signal","Szintillator Signal verstärkt"][i])
    plt.ylabel("Spannung [V]")
    plt.xlabel("Zeit [ns]")
    plt.show()

In [ ]:
u,du=np.array(Energie[["u","d_u"]]).T
[m,n],[dm,dn],chi=Ge_Energie_Cal
E=(u-n)/m
dE=np.sqrt((du/m)**2+(dn/m)**2+((u-n)/m**2*dm)**2)
print(E)
print(dE)